In [1]:
using BenchmarkTools
mode = ["normal", "@inbounds"] 
i= 0
for inbounds in ["", "@inbounds"]
       global i += 1
       eval(Meta.parse("""function f$i(x::AbstractArray{<:Real})
                              y= 0
                              $inbounds for i in eachindex(x)
                                  y += x[i] > 0.5
                              end
                              y 
                          end"""))
end

x = rand(10^7)
for (idx, f) in enumerate([f1, f2])
    println("\n", mode[idx])
    @btime $f($x)
end


normal
  7.005 ms (0 allocations: 0 bytes)

@inbounds
  3.627 ms (0 allocations: 0 bytes)


In [2]:
run(`julia inbounds.jl`)


normal
  6.994 ms (0 allocations: 0 bytes)

@inbounds
  3.629 ms (0 allocations: 0 bytes)


Process(`julia inbounds.jl`, ProcessExited(0))

In [3]:
run(`julia --check-bounds=yes inbounds.jl`)


normal
  6.998 ms (0 allocations: 0 bytes)

@inbounds
  7.003 ms (0 allocations: 0 bytes)


Process(`julia --check-bounds=yes inbounds.jl`, ProcessExited(0))

In [4]:
run(`julia --check-bounds=no inbounds.jl`)


normal
  3.756 ms (0 allocations: 0 bytes)

@inbounds
  3.592 ms (0 allocations: 0 bytes)


Process(`julia --check-bounds=no inbounds.jl`, ProcessExited(0))

In [5]:
x = [1,2,3]

3-element Array{Int64,1}:
 1
 2
 3

In [6]:
x[4]

BoundsError: BoundsError: attempt to access 3-element Array{Int64,1} at index [4]

In [7]:
f(x) = @inbounds x[1]

f (generic function with 1 method)

In [8]:
g(x) = x[1]

g (generic function with 1 method)

In [9]:
@code_native f([1])

	.text
; ┌ @ In[7]:1 within `f'
; │┌ @ In[7]:1 within `getindex'
	movq	(%rdi), %rax
	movq	(%rax), %rax
; │└
	retq
	nopw	(%rax,%rax)
; └


In [10]:
@code_native g([1])

	.text
; ┌ @ In[8]:1 within `g'
; │┌ @ In[8]:1 within `getindex'
	cmpq	$0, 8(%rdi)
	je	L14
	movq	(%rdi), %rax
	movq	(%rax), %rax
; │└
	retq
L14:
	pushq	%rbp
	movq	%rsp, %rbp
; │ @ In[8]:1 within `g'
; │┌ @ array.jl:728 within `getindex'
	movq	%rsp, %rax
	leaq	-16(%rax), %rsi
	movq	%rsi, %rsp
	movq	$1, -16(%rax)
	movabsq	$jl_bounds_error_ints, %rax
	movl	$1, %edx
	callq	*%rax
	nopw	%cs:(%rax,%rax)
; └└


In [11]:
@inline function getindex(B::BitArray, i::Int)
    @boundscheck checkbounds(B, i)
    unsafe_bitgetindex(B.chunks, i)
end

getindex (generic function with 1 method)